# Tutorial: Cognitive Service Vision Product Recognition using Python

This is a tutorial about using Cognitive Service Vision Product Recognition.

Cognitive Service Vision Product Recognition can be used to detect and recognize packaged products.

Currently, product recognition feature are available in **EastUS**, **West US2**, and **West Europe** regions.

Please create a computer vision resource on Azure portal, in **EastUS**, **West US2**, or **West Europe** region, if you don't already have one. You can use [Multi-service resource](https://learn.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Canomaly-detector%2Clanguage-service%2Ccomputer-vision%2Cwindows) as well. 

![check ../model_customization/media/create_cv_resource.png if pic does not show up](../model_customization/media/create_cv_resource.png)

## Install the python samples package

Install the sample code including utility code helping you use Python to run product recognition in this tutorial.

In [ ]:
pip install cognitive-service-vision-model-customization-python-samples

In [ ]:
# Resource and key
import logging
import uuid
logging.getLogger().setLevel(logging.INFO)

import cv2
import numpy as np
from PIL import Image

from cognitive_service_vision_model_customization_python_samples import ResourceType
from cognitive_service_vision_model_customization_python_samples.clients import ProductRecognitionClient
from cognitive_service_vision_model_customization_python_samples.models import ProductRecognition
from cognitive_service_vision_model_customization_python_samples.tools import visualize_recognition_result

## Credentials

Resource name and resource key are needed for accessing the service, which you can find here:

![check media/credentials.png if pic does not show up](./resources/credentials.png)

If you are using a multi-service resource, the endpoint can be found here:

![check resources/multi_service_endpoint.png if pic does not show up](./resources/multi_service_endpoint.png)

In [ ]:
# Resource and key
resource_type = ResourceType.SINGLE_SERVICE_RESOURCE # or ResourceType.MULTI_SERVICE_RESOURCE

resource_name = None
multi_service_endpoint = None

if resource_type == ResourceType.SINGLE_SERVICE_RESOURCE:
    resource_name = '{specify_your_resource_name}'
    assert resource_name
else:
    multi_service_endpoint = '{specify_your_service_endpoint}'
    assert multi_service_endpoint

resource_key = '{specify_your_resource_key}'

## Pre-process your image using the image composition tool (Optional)

Follow the tutorial [here](./cognitive_service_vision_image_composition.ipynb) to stitch and rectify your images.
Stitching is needed when the shelf is too long or too high so that one image capture cannot cover the entire shelf.
Rectification warps the image so that it looks frontal, it is required for doing planogram compliance. For product recognition, it's optional but recommended, as it will improve recognition quality.

## Run product recognition with pre-built model

You can run product recognition to detect products and gaps using our prebuilt model. The prebuilt model only detects whether an object is a product or a gap, it does not recognize different names of the products. You can use the prebuilt model to check scenarios like shelf occupancy rate.

In [ ]:
client = ProductRecognitionClient(resource_type, resource_name, multi_service_endpoint, resource_key)
run_name = str(uuid.uuid4())
model_name = 'ms-pretrained-product-detection'
run = ProductRecognition(run_name, model_name)

with open('./resources/sample_image.jpg', 'rb') as f:
    img = f.read()

try:
    client.create_run(run, img, 'image/png')
    result = client.wait_for_completion(run_name, model_name)
finally:
    client.delete_run(run_name, model_name)

## Visualize product recognition result from pre-built model

In [ ]:
# Convert img data to numpy array and decode numpy array to OpenCV BGR image
cv_img = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)

# Visualize the result
cv_img = visualize_recognition_result(result.result, cv_img)
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
display(Image.fromarray(cv_img))

## Run product recognition with customized product recognition model

To get more detailed recognition result from the image, you will need to train a customized product recognition model. So that the model can recognize different types of product, using which inventory check, planogram compliance and other more detailed retail execution KPIs can be calculated.

### Train customized product recognition model

Follow the [Cognitive Service Vision Model Customization document](https://github.com/Azure-Samples/cognitive-service-vision-model-customization-python-samples/blob/main/docs/cognitive_service_vision_model_customization.ipynb) to train a customized product recognition model

Note that for product recognition model training, the dataset type needs to be **ObjectDetection**, and the model kind needs to be **Product-Recognizer**

Once you finished training the model, replace the corresponding model_name in the below code cell.

In [ ]:
client = ProductRecognitionClient(resource_type, resource_name, multi_service_endpoint, resource_key)
run_name = str(uuid.uuid4())
model_name = '{specify_your_customized_product_recognition_model_name}'
run = ProductRecognition(run_name, model_name)

with open('./resources/sample_image.jpg', 'rb') as f:
    img = f.read()

try:
    client.create_run(run, img, 'image/png')
    result = client.wait_for_completion(run_name, model_name)
finally:
    client.delete_run(run_name, model_name)

## Visualize product recognition result from customized model

In [ ]:
# Convert img data to numpy array and decode numpy array to OpenCV BGR image
cv_img = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)

# Visualize the result
cv_img = visualize_recognition_result(result.result, cv_img)
cv_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
display(Image.fromarray(cv_img))

## Next steps

With the accurate product recognition result, users can now effectively carry out various retail execution key performance indicators (KPIs). One essential KPI that can be measured is shelf occupancy, which involves monitoring the amount of shelf space occupied by specific products. By leveraging the product recognition outcome, retailers can analyze and optimize the distribution of products on the shelves, ensuring maximum visibility and availability.

Additionally, for those who have a planogram in place, they can easily assess planogram compliance using the product recognition data, please refer to the tutorial [here](cognitive_service_vision_planogram_compliance.ipynb)